## Recap
- Everything is an object in Python, i.e., an instance of a class. Even classes are objects of class `type`!
- Instance Variables/Attributes comprise the state of an object
- Instance Methods/Functions are used to change the state of the object

![](https://pythonschool.net/oop/images/field_class_methods_diagram2.png)
- You access an instance variable x of object o by doing o.x
- Each object contains a dictionary that stores its attributes. These attributes can be accessed through the attribute `__dict__`
- There are no interfaces in Python! In fact with duck typing, you indeed don’t need the ability of interfaces to group together unrelated classes

### Tuples vs. Objects

In [0]:
def my_func(arg):
    print(arg)

E = [('1', '2'), ('1', '3')]
V = set(['1', '2', '3'])

G = (E, V, my_func)

print(G[1])
G[2]('test')

text = 'new str'

G = (E, text, V, my_func)
print(G[1])

## Encapsulation
The main utility of Object-Orientation is to encapsulate related
data behind a well-defined interface through which the object can be manipulated

In [0]:
# class Student(object):
class Student:
    '''
    Student class
    
    Doc tests:
    
    first_student = Student("John", 22)
    print(first_student.name)
    >>> John
    
    first_student.add_classmates([Student("Sue", 22), Student("Jack", 22)])
    print([o.name for o in first_student._classmates])
    >>> ['Sue', 'Jack']
    '''

    def __init__(self, name, age, classmates = None):
        '''
        This method is invoked after the Student() constructor method
        __new__ is invoked to instantiate a new instance of class Student.
        self is bound to the newly created bare object, and then
        __init__ initializes the initial state of this object.
        '''
        
        
        # Inside a method, you access an instance variable x using self.x
        self.name = name
        
        # The leading-underscore convention: Variable names 
        # starting with a single underscore are intended to be private. 
        # But you are not prevented from touching them from outside.
        if classmates is None:
            self._classmates = []
        else:
            self._classmates = classmates
            
        # We may want to check on construction that all classmates are students
        if not all([ type(std) is Student for std in self._classmates ]):
            raise ValueError("Not every classmate is a student")
            
        # Name mangling: a mechanism to prevent accidental access to
        # instance variables defined by derived classes
        self.__age = age
        
    def add_classmates(self, classmates):
        '''
        This method adds instances of class Student
        to the list of classmates of this object, 
        raising an exception if input argument is not 
        a list of instances of class Student
        '''
        
        if not all([ type(std) is Student for std in classmates ]):
            raise ValueError("Not every classmate is a student")
        else:
            self._classmates.extend(classmates)

In [0]:
# create a new instance of class Student
new_student = Student("John", 22)

### Name Mangling
Any identifier of the form `__var` (at least two leading underscores, at most one trailing underscore) is textually replaced with `_classname__var`, where classname is the current class name with leading underscore(s) stripped.

This prevents from accidentally overriding the private methods and attributes of the superclass.

In [3]:
# list all (key, value) pairs in the object's dictionary
print(new_student.__dict__)

{'name': 'John', '_classmates': [], '_Student__age': 22}


Encapsulation is broken in Python due to the lack of support for information hidding

In [4]:
new_student.name = "Jack"
print(new_student.name)

# You can access the private attribute though!
print(new_student._classmates)
print(new_student._Student__age)

Jack
[]
22


In [0]:
new_student._Student__age = 18
print(new_student.__dict__)

{'name': 'Jack', '_classmates': [], '_Student__age': 18}


In [0]:
# This creates a new instance attribute
new_student.__age = 20
print(new_student.__dict__)

{'name': 'Jack', '_classmates': [], '_Student__age': 18, '__age': 20}


In [0]:
new_student.add_classmates([Student("Sue", 22), Student("Jack", 22)])
print([o.name for o in new_student._classmates])

['Sue', 'Jack']


In [0]:
new_student.add_classmates("Tom")

ValueError: Not every classmate is a student

In [0]:
print(new_student.__dict__)

{'name': 'Jack', '_classmates': [<__main__.Student object at 0x103f36160>, <__main__.Student object at 0x103f36f98>], '_Student__age': 18, '__age': 20}


__Other ways to read or update instance variables?__<br>
The interface provides Accessors to the instance variables. Typically they are called get and set. Set can change the state, and is thus an example of a Mutator.

In [0]:
class Student:

    def __init__(self, name, age):
        self.name = name
        self.__age = age
        
    def setAge(self, age):
        self.__age = age
        
    def getAge(self):
        return self.__age
    

new_student = Student("John", 22)
print(new_student.__dict__)
print(new_student.name)

Overwritting object's `__getattribute__` and `__setattr__` methods

In [0]:
class Student:

    def __init__(self, name, age):
        self.name = name
        self.__age = age

    def __getattribute__(self, attr):
        if attr == "_Student__age":
            print("Error: This is a private attribute and shouldn't be accessed from outside!")
            # raise AttributeError
            
        return object.__getattribute__(self, attr)

    def __setattr__(self, attr, value):
        if attr == "_Student__age":
            print("Error: This is a private attribute and shouldn't be accessed from outside!")
            # raise AttributeError
        
        # object is the superclass of all python classes
        object.__setattr__(self, attr, value)
            
            
new_student = Student("John", 22)
print(new_student.__dict__)
print(new_student.name)

Error: This is a private attribute and shouldn't be accessed from outside!
{'name': 'John', '_Student__age': 22}
John


In [0]:
print(new_student._Student__age)

Error: This is a private attribute and shouldn't be accessed from outside!
22


## Motivating Inheritance
Inheritance provides a mechanism that allows us to build on our existing work (reusing and extending). The relationship between classes benefiting from inheritance is similar to that of a parent and child.

In [0]:
import math

class Shape:
    def __init__(self, color, filled):
        self.__color = color
        self.__filled = filled

    def get_color(self):
        return self.__color

    def set_color(self, color):
        self.__color = color

    def get_filled(self):
        return self.__filled

    def set_filled(self, filled):
        self.__filled = filled

        
class Rectangle:
    def __int__(self, color, filled, width, length):
        self.__color = color
        self.__filled = filled
        self.__width = width
        self.__length = length

    def get_color(self):
        return self.__color

    def set_color(self, color):
        self.__color = color

    def is_filled(self):
        return self.__filled

    def set_filled(self, filled):
        self.__filled = filled

    def get_area():
        return self.__width * self.__length

    
class Circle:
    def __int__(self, color, filled, radius):
        self.__color = color
        self.__filled = filled
        self.__radius = radius      

    def get_color(self):
        return self.__color

    def set_color(self, color):
        self.__color = color

    def is_filled(self):
        return self.__filled

    def set_filled(self, filled):
        self.__filled = filled

    def get_area(self):
        return math.pi * self.__radius ** 2

One thing you notice when building Rectangle is that a Rectangle is just
like a Shape, but has some additional state and behaviour.  In this
sense: a Rectangle behaves-like-a Shape with respect to Shape behaviour.
That is, where ever we had an algorithm that operated with Shape, it 
should operate exactly the same when using Rectangle.  Or said another
way, noting in the interface and behaviour of Rectangle should violate
Rectangle-ness.

So we would like to reuse all the state and behaviour of Shape, and
add some new state and behaviour. Plus we would like to have this reuse
extensible, in the sense that if we add behaviour to Shape, such as
with new methods, we would like to get this for free in Rectangle.

## Inheritance
Inheritance allows the programmer to define new types based on existing types. Python has an inheritance model.

Inheritance allows programmer to create a general class first then later extend it to more specialized class. Hence, it allows programmer to write better code.

![class diagram](https://pythonschool.net/oop/images/animal_class_inheritance_diagram.png)


### Syntax
`class Derived(Base):`

defines the new type `Derived` based on `Base`. Here, `Base` must be the name of an existing type (used defined, or built-in). In the parenthesis one can list more than one types, in which case Derived inherits from all of them. This is called multiple-inheritance which will be covered later.

In [0]:
class MyInt(int):
    pass

In Python, all classes inherits from the object class implicitly. Thus, in the absence of any superclasses that a class inherits from, the superclass is always __object__

In [0]:
# These are equivalent
class ParentClass:
    pass

class ParentClass(object):
    pass

### Special Class Members

It turns out that the object class provides some special methods with two leading and trailing underscores which are inherited by all the classes. Here are some important methods provided by the object class.

`__new__()
__init__()
__str__()`

The `__new__()` method creates the object. After creating the object it calls the `__init__()` method to initialize attributes of the object. Finally, it returns the newly created object to the calling program. Normally, we don't override `__new__()` method, however if you want to significantly change the way an object is created, you should definitely override it.

The `__str__()` method is used to return a nicely formatted string representation of the object. The object class version of `__str__()` method returns a string containing name of the class and its memory address in hexadecimal. For example:

In [6]:
class MyClass:
    def __str__(self):
        return "A more helpful description"

obj = MyClass()
print(obj)

A more helpful description


In [0]:
class ParentClass:
    # body of ParentClass
    # instance methods and attributes
    pass

class ChildClass(ParentClass):
    # body of ChildClass
    # instance methods and attributes
    pass

class GrandChildClass(ChildClass):
    # body of GrandChildClass
    # instance methods and attributes
    pass

The classes that a new class is derived from are called either the base classes, or superclasses (or base class/superclass, when there is only one such class). The derived class is often called a subclass. The superclass is also called the parent class, and the superclass of the superclass the grandparent class, etc. Subclassing is when we use inheritance to derive one class from another.

In [0]:
print(ParentClass.__base__)
print(ChildClass.__base__)
print(GrandChildClass.__base__)

<class 'object'>
<class '__main__.ParentClass'>
<class '__main__.ChildClass'>


### Namespaces and Classes

When Python executes the class definition, it first executes the statements within the class definition in a new execution frame (with a local namespace, as if we were in a function) and when this execution finishes, it creates an object of type type whose namespace is created from the local namespace that resulted from executing the statements in the class definition. The created object will represent the class just defined. Finally, Python binds the class' name in the current namespace to this object. As a result of all this, the object's namespace holds the definitions of the class-level attributes (class level variables and methods). We call the namespace of the resulting object the class's namespace.

__Exercise__: Write two classes called Rectangle and Circle by extending Shape. 
These classes should have new attributes and methods to calculate their area and perimeter

In [0]:
import math

class Shape:
    def __init__(self, color='black', filled=False):
        self.__color = color
        self.__filled = filled

    def get_color(self):
        return self.__color

    def set_color(self, color):
        self.__color = color

    def get_filled(self):
        return self.__filled

    def set_filled(self, filled):
        self.__filled = filled


class Rectangle(Shape):
    def __init__(self, length, breadth, color='black', filled=False):
        # Calling base class constructor from child class constructor
        super().__init__(color, filled)
        
        self._length = length
        self._breadth = breadth
    
    def get_length(self):
        return self._length
    
    def get_area(self):
        return self._length*self._breadth

    def get_perimeter(self):
        return 2*(self._length+self._breadth)


class Circle(Shape):
    def __init__(self, radius, color='black', filled=False):
        # Calling base class constructor from child class constructor 
        super().__init__(color, filled)
        
        self._radius = radius

    def get_area(self):
        return math.pi * self._radius ** 2

    def get_perimeter(self):
        return 2* math.pi * self._radius

In [0]:
class MyRedCircle(Circle):
    def __init__(self, radius):
        # Calling base class constructor from child class constructor 
        super().__init__(radius)
        
        # Calling base class method from child class
        super().set_color('red')

In [17]:
r1 = Rectangle(10.5, 2.5)
print("Type of r1:", type(r1))
print("Area of rectangle r1:", r1.get_area())
print("Perimeter of rectangle r1:", r1.get_perimeter())
print("Color of rectangle r1:", r1.get_color())
print("Is rectangle r1 filled ? ", r1.get_filled())
r1.set_filled(True)
print("Is rectangle r1 filled ? ", r1.get_filled())
r1.set_color("orange")
print("Color of rectangle r1:", r1.get_color())

c1 = Circle(12)
print("Type of c1:", type(c1))
print("\nArea of circle c1:", format(c1.get_area(), "0.2f"))
print("Perimeter of circle c1:", format(c1.get_perimeter(), "0.2f"))
print("Color of circle c1:", c1.get_color())
print("Is circle c1 filled ? ", c1.get_filled())
c1.set_filled(True)
print("Is circle c1 filled ? ", c1.get_filled())
c1.set_color("blue")
print("Color of circle c1:", c1.get_color())


c2 = MyRedCircle(10)
print("Type of c2:", type(c2))
print("\nArea of circle c2:", format(c2.get_area(), "0.2f"))
print("Perimeter of circle c2:", format(c2.get_perimeter(), "0.2f"))
print("Color of circle c2:", c2.get_color())
print("Is circle c2 filled ? ", c2.get_filled())
c2.set_filled(True)
print("Is circle c2 filled ? ", c2.get_filled())
c2.set_color("blue")
print("Color of circle c2:", c2.get_color())

Type of r1: <class '__main__.Rectangle'>
Area of rectangle r1: 26.25
Perimeter of rectangle r1: 26.0
Color of rectangle r1: black
Is rectangle r1 filled ?  False
Is rectangle r1 filled ?  True
Color of rectangle r1: orange
Type of c1: <class '__main__.Circle'>

Area of circle c1: 452.39
Perimeter of circle c1: 75.40
Color of circle c1: black
Is circle c1 filled ?  False
Is circle c1 filled ?  True
Color of circle c1: blue
Type of c2: <class '__main__.MyRedCircle'>

Area of circle c2: 314.16
Perimeter of circle c2: 62.83
Color of circle c2: red
Is circle c2 filled ?  False
Is circle c2 filled ?  True
Color of circle c2: blue
